In [1]:
import sys
sys.path.append('/home/mount/project/')
sys.path.append('Neural_Network_DNA_Demo/')

In [2]:
from config_sing import *
from collections import Counter
from helper import IOHelper, SequenceHelper

CHROMS = np.r_[np.arange(1,23), ["X"]]

## Import peak sequence

In [3]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"

lst = list()
for chrom in CHROMS:
    ###
    fname = "chr" + chrom + ".fa"
    fpath = os.path.join(fdiry, fname)
    print(fpath, flush = True)
    dat = IOHelper.get_fastas_from_file(fpath, uppercase=True)
    
    ###
    lst.append(dat)

dat_fasta = pd.concat(lst).reset_index()
dat_fasta.head()

/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr1.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr2.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr3.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr4.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr5.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr6.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr7.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr8.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr9.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr10.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr11.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr12.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data/chr13.fa
/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_d

,index,location,sequence
0,0,chr1:778232-779232,TCCCTGTTGCACACCTGGATAGTTTTTTAAATTTTTTTAGACAGGG...
1,1,chr1:818441-819441,ATGCTTACCAGGGCTGGAGGCCCCTGTGAGTAGTGAAGTGCATGTG...
2,2,chr1:826644-827644,ATTCTGTCCTCCATATCTCTGCTGATTCCCATTTTGTCTATTTCCA...
3,3,chr1:851117-852117,TGTCTTTATAGCAGCATGATTTATAGTCCTTTGGGTATATACCCAG...
4,4,chr1:856205-857205,CATTCCCGGATCCAAGGTCCTGCCTCTGTCCTGTCCCCTGAGCCAC...


**Check**

In [4]:
tmp = dat_fasta.sequence.apply(lambda seq: len(seq))
Counter(tmp)

Counter({1000: 87909})

## One hard encoding sequence

In [5]:
sequence = dat_fasta.sequence
sequence_length = len(dat_fasta.sequence[0])

In [5]:
# Convert sequence to one hot encoding matrix
seq_matrix = SequenceHelper.do_one_hot_encoding(
    sequence, 
    sequence_length,
    SequenceHelper.parse_alpha_to_seq)

**arrange and reshape**

In [16]:
X = np.nan_to_num(seq_matrix) # Replace NaN with zero and infinity with large finite numbers
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

## Import peak activity

In [7]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results"
lst = list()

for chrom in CHROMS:
    ###
    fname = "dat_score_chr" + chrom + ".tsv"
    fpath = os.path.join(fdiry, fname)
    print(fpath)
    dat = pd.read_table(fpath)
    
    ###
    lst.append(dat)

dat_activity = pd.concat(lst)
print(dat_activity)

/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr1.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr2.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr3.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr4.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr5.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr6.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr7.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr8.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr9.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr10.tsv
/home/mount/work/out/proj_combeffect/peak/cradle_peak_count/results/dat_score_chr11.tsv
/home/mount/work/out/proj_combeffect/peak

**Check**

In [9]:
dat_fasta.location

0              chr1:778232-779232
1              chr1:818441-819441
2              chr1:826644-827644
3              chr1:851117-852117
4              chr1:856205-857205
                   ...           
87904    chrX:155434820-155435820
87905    chrX:155435206-155436206
87906    chrX:155440330-155441330
87907    chrX:155521212-155522212
87908    chrX:155521255-155522255
Name: location, Length: 87909, dtype: object

In [14]:
dat_activity.Name

0             chr1:778588-778877:DMSO
1              chr1:818351-819531:Dex
2             chr1:826715-827573:DMSO
3             chr1:851268-851966:DMSO
4              chr1:856338-857072:Dex
                    ...              
3426     chrX:155435035-155435606:Dex
3427    chrX:155435256-155436156:DMSO
3428    chrX:155440404-155441256:DMSO
3429     chrX:155521256-155522169:Dex
3430    chrX:155521377-155522133:DMSO
Name: Name, Length: 87909, dtype: object

**arrange and reshape**

In [15]:
Y_dmso = dat_activity.Log2_Score_DMSO
Y_dex  = dat_activity.Log2_Score_Dex
Y = np.array([Y_dmso, Y_dex])
Y = Y.T

## Store data

In [ ]:
print(sequence.shape)
print(X.shape)
print(Y.shape)

In [20]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"

fname = "whole_genome_X.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, X)

fname = "whole_genome_Y.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, Y)

In [6]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"
fname = "whole_genome_Seq.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, sequence)

## Subset data

In [1]:
import sys
sys.path.append('/home/mount/project/')
from config_sing import *
from sklearn.model_selection import train_test_split

In [2]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"

fname = "whole_genome_X.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'rb') as file:
    X = np.load(file)

fname = "whole_genome_Y.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'rb') as file:
    Y = np.load(file)
    
fname = "whole_genome_Seq.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'rb') as file:
    S = np.load(file, allow_pickle=True)

In [13]:
X[0,:20]

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [14]:
S[0][:20]

'TCCCTGTTGCACACCTGGAT'

In [15]:
X_train, X_valid, y_train, y_valid, S_train, S_valid = train_test_split(
    X, 
    Y, 
    S,
    test_size=0.2, 
    random_state=123)

In [16]:
print(X.shape)
print(Y.shape)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(87909, 1000, 4)
(87909, 2)
(70327, 1000, 4)
(17582, 1000, 4)
(70327, 2)
(17582, 2)


In [18]:
np.random.seed(123)
n     = 10000
idx   = np.random.choice(X_train.shape[0], size = n)
X_sub = X_train[idx]
y_sub = y_train[idx]
S_sub = S_train[idx]
print(X_sub.shape)
print(y_sub.shape)
print(S_sub.shape)

(10000, 1000, 4)
(10000, 2)
(10000,)


In [19]:
fdiry = "/home/mount/work/out/proj_combeffect/peak/cradle_deepstarr_data"

fname = "whole_genome_X_sub.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, X_sub)

fname = "whole_genome_Y_sub.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, y_sub)
    
fname = "whole_genome_Seq_sub.npy"
fpath = os.path.join(fdiry, fname)
with open(fpath, 'wb') as file:
    np.save(file, S_sub)